# Resilient Distributed Datasets (RDDs)

*Resilient Distributed Dataset* (RDD) est une collection d'objets immuables et distribués. Les RDD sont résilients ou tolérants aux pannes. Les collections d'objets partitionnés sont réparties dans un cluster, stockées en mémoire ou sur disque. Les RDD sont construits et manipulés grâce à un ensemble diversifié de transformations parallèles (*map*, *filter*, *join*) et d'actions (*count*, *collect*, *save*)

Un RDD peut être créé de plusieurs manières:
* Paralléliser une collection
* Lire des données à partir d'une source externe
* Transformation d'un RDD existant
* API de streaming

## RDD  Operations

###  Transformation (Lazy evaluation)
Les transformations créent de nouvelles dataset à partir d'une dataset existante. Par exemple, *map* est une transformation qui passe chaque élément de l'ensemble de données via une fonction et renvoie un nouveau RDD représentant les résultats. Toutes les transformations dans Spark sont *lazy*, c'est à dire elles ne calculent pas le résultats automatiquement. Les transformations ne sont calculées que lorsqu'une action nécessite qu'un résultat soit renvoyé.  

**Exemple de transformations**

*map(),  flatMap(), filter(),  distinct(), intersection(), cartesian(), groupByKey(), coalesce(), mapPartitions(), reduceByKey(), repartition(), sortByKey(), partitionBy(), sample(), ...*  

### Actions
Les actions permettent de returner la valeur calculer sur le dataset vers le driver. Par exemple, *reduce* est une action qui agrège tous les éléments du RDD en utilisant une fonction et renvoie le résultat final vers le driver.

**Exemple d'actions**
*reduce(), collect(), count(), first(), take() countByKey(), takeSample(), foreach(), takeOrdered(), saveAsTextFile(), saveAsSequenceFile() ...*

![](images/rdd.png)

**Note** : Dans ce chapitre nous allons faire une introduction des RDD pour vous permettre de comprendre comment fonctionne Spark built-in. Depuis les récentes versions de Spark,les RDD sont désormais considérés dans API de bas niveau.

#### Parralleliser un collection

In [ ]:
val rdd = sc.parallelize(Seq(0,1,2,3,4,5,6,7,8,9))

In [ ]:
rdd

In [ ]:
rdd.take(10)

In [ ]:
rdd.take(10).foreach(println)

In [ ]:
val rdd_filter = rdd.filter(x => x % 2 == 0)

In [ ]:
rdd.filter(x => x % 2 == 0).take(5)

In [ ]:
rdd_filter.collect()

In [ ]:
val rdd_map = rdd_filter.map(x => x*x)

In [ ]:
rdd_map.take(10)

In [ ]:
val result = rdd_map.reduce((a, b) => a+b)

In [ ]:
// Foctorisation du code 
// piplining
val sum_even_sqr =  rdd.filter(x => x % 2 == 0).map( x => x*x).map(x => x*x*x).reduce((x,y) => x + y)

### Reading Files

In [1]:
val texte= sc.textFile("datasets/senegal.txt")

Intitializing Scala interpreter ...

Spark Web UI available at http://172.17.0.1:4042
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1608723679231)
SparkSession available as 'spark'


texte: org.apache.spark.rdd.RDD[String] = datasets/senegal.txt MapPartitionsRDD[1] at textFile at <console>:25


In [2]:
//Numbre de ligne
texte.count

res0: Long = 5


In [3]:
texte.first

res1: String = "Le Sénégal est un pays situé sur la côte ouest de l'Afrique et doté d'un héritage colonial français et de nombreuses attractions naturelles. "


#### Preprocessing sur un fichier texte

In [4]:
// convertir du texte en MAJ
val fistLines = texte.map(line => line.toUpperCase()).take(2)

fistLines: Array[String] = Array("LE SÉNÉGAL EST UN PAYS SITUÉ SUR LA CÔTE OUEST DE L'AFRIQUE ET DOTÉ D'UN HÉRITAGE COLONIAL FRANÇAIS ET DE NOMBREUSES ATTRACTIONS NATURELLES. ", "DAKAR, LA CAPITALE, COMPREND LE QUARTIER HISTORIQUE DE LA MÉDINA ET LE CÉLÈBRE MUSÉE THÉODORE MONOD, EXPOSANT DES ŒUVRES D'ART AFRICAIN. ")


In [5]:
fistLines(0)

res2: String = "LE SÉNÉGAL EST UN PAYS SITUÉ SUR LA CÔTE OUEST DE L'AFRIQUE ET DOTÉ D'UN HÉRITAGE COLONIAL FRANÇAIS ET DE NOMBREUSES ATTRACTIONS NATURELLES. "


In [6]:
// filtre les ligne commençant par L
texte.filter(line => line.startsWith("L")).take(4).foreach(println)

Le Sénégal est un pays situé sur la côte ouest de l'Afrique et doté d'un héritage colonial français et de nombreuses attractions naturelles. 
Le Sénégal est un pays dont le président est démocratiquement élu au suffrage universel directe.


In [7]:
texte.map(line => line.toUpperCase()).filter(line => line.startsWith("L")).take(4).foreach(println)

LE SÉNÉGAL EST UN PAYS SITUÉ SUR LA CÔTE OUEST DE L'AFRIQUE ET DOTÉ D'UN HÉRITAGE COLONIAL FRANÇAIS ET DE NOMBREUSES ATTRACTIONS NATURELLES. 
LE SÉNÉGAL EST UN PAYS DONT LE PRÉSIDENT EST DÉMOCRATIQUEMENT ÉLU AU SUFFRAGE UNIVERSEL DIRECTE.


In [8]:
val rdd = texte.flatMap(line => line.split(' ')).distinct()

rdd: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[9] at distinct at <console>:26


In [9]:
rdd.take(10).foreach(println)

à
sur
française,
directe.
pour
des
ouest
comprend
d'un
coloniale.


### zip

In [11]:
val capitales = sc.parallelize(Seq("Abuja","Accra", "Addis-Abeba", "Alger", "Dakar", "Banjul", 
                                   "Antananarivo", "Bamako", "Bissau","Caire"))

capitales: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[10] at parallelize at <console>:25


In [12]:
capitales.take(5)

res7: Array[String] = Array(Abuja, Accra, Addis-Abeba, Alger, Dakar)


In [13]:
val pays =  sc.parallelize(Seq("Nigeria", "Ghana", "Éthiopie", "Algérie", "Sénégal", "Gambie", 
                               "Madagascar","Mali","Guinée-Bissau", "Égypte"))

pays: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[11] at parallelize at <console>:25


In [14]:
val pays_capitales = pays.zip(capitales)

pays_capitales: org.apache.spark.rdd.RDD[(String, String)] = ZippedPartitionsRDD2[12] at zip at <console>:28


In [15]:
 pays_capitales.collect().foreach(println)

(Nigeria,Abuja)
(Ghana,Accra)
(Éthiopie,Addis-Abeba)
(Algérie,Alger)
(Sénégal,Dakar)
(Gambie,Banjul)
(Madagascar,Antananarivo)
(Mali,Bamako)
(Guinée-Bissau,Bissau)
(Égypte,Caire)


### Le famous wordcount avec Spark (RDD)

In [16]:
val rdd_figaro =  sc.textFile("datasets/ancien_figaro.txt")

rdd_figaro: org.apache.spark.rdd.RDD[String] = datasets/ancien_figaro.txt MapPartitionsRDD[14] at textFile at <console>:25


In [17]:
val rdd_flatmap = rdd_figaro.flatMap(line => line.split(" "))

rdd_flatmap: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[15] at flatMap at <console>:26


In [18]:
val rdd_map = rdd_flatmap.map(w => (w, 1))

rdd_map: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[16] at map at <console>:26


In [19]:
val wordcounts = rdd_map.reduceByKey((w1, w2) => w1 + w2)

wordcounts: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[17] at reduceByKey at <console>:26


In [20]:
wordcounts

res9: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[17] at reduceByKey at <console>:26


In [21]:
wordcounts.take(10).foreach(println)

(journée,,1)
(Ah!,19)
(apostrophes,2)
(destitué;,1)
(budget!,2)
(souvent,22)
(combinaison,,1)
(épigrammes.,2)
(plaisir,,2)
(resteront,1)


In [24]:
// Forme condensée
val wordcounts = rdd_figaro.flatMap(_.split(" ")).map((_, 1)).reduceByKey(_+_)   

wordcounts: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[23] at reduceByKey at <console>:27


In [25]:
wordcounts.take(10).foreach(println)

(journée,,1)
(Ah!,19)
(apostrophes,2)
(destitué;,1)
(budget!,2)
(souvent,22)
(combinaison,,1)
(épigrammes.,2)
(plaisir,,2)
(resteront,1)


### Exercice  

Reprendre le wordcount en supprimant les ponctuations, rendre le texte en minuscule, et ordonner le resultat en ordre décroissant des mots.